In [ ]:
import paho.mqtt.client as mqtt
import mysql.connector
import time
from datetime import datetime

# MQTT Broker configuration
mqtt_broker_address = "broker_address"
mqtt_broker_port = 1883
mqtt_username = "username"
mqtt_password = "password"

# MySQL Database configuration
mysql_host = "host_address"
mysql_user = "username"
mysql_password = "password"
mysql_database = "database_name"

# MQTT on_connect callback
def on_connect(client, userdata, flags, rc):
    print("Connected to MQTT broker with result code " + str(rc))
    # Subscribe to the topics
    client.subscribe("topic1")
    client.subscribe("topic2")
    client.subscribe("topic3")

# MQTT on_message callback
def on_message(client, userdata, message):
    # Parse the received message payload
    payload = message.payload.decode("utf-8")
    topic = message.topic
    
    # Store the data to MySQL database
    try:
        cnx = mysql.connector.connect(user=mysql_user, password=mysql_password,
                                      host=mysql_host, database=mysql_database)
        cursor = cnx.cursor()
        add_data = ("INSERT INTO table_name (timestamp, topic, payload) VALUES (%s, %s, %s)")
        data = (datetime.now(), topic, payload)
        cursor.execute(add_data, data)
        cnx.commit()
        cursor.close()
        cnx.close()
        print("Data added to MySQL database: " + topic + " " + payload)
    except mysql.connector.Error as err:
        print("Failed to add data to MySQL database: {}".format(err))

# MQTT client configuration
client = mqtt.Client()
client.username_pw_set(username=mqtt_username, password=mqtt_password)
client.on_connect = on_connect
client.on_message = on_message
client.connect(mqtt_broker_address, mqtt_broker_port)

# MQTT client loop
while True:
    client.loop_start()
    time.sleep(900)  # wait for 15 minutes
    client.loop_stop()
